In [ ]:
import requests
res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")


In [ ]:
import pandas as pd

df = pd.read_html(res.text)[0]
df

In [ ]:
# 設定column名稱
df.columns = df.iloc[0]
# 刪除第一行
df = df.iloc[1:]
df

In [ ]:
# 先移除row，再移除column，超過三個NaN則移除
df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
df.iloc[9976,:]

In [ ]:

adress = df.iloc[0:941,0]
adress

In [ ]:
import numpy as np
adress = np.array(adress)
number = [ i[0:4] for i in adress]
name = [ i[5:] for i in adress]

In [ ]:
import datetime
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
start = datetime.datetime(2009, 1, 1)
end = datetime.datetime(2020, 1, 1)
data = web.DataReader('2330.TW', 'yahoo', start, end)

In [ ]:
data

In [ ]:
data.plot()

In [ ]:
data.shape[1]

In [ ]:
#套件
import pymysql
#設定 DB 資料 
db = pymysql.connect("localhost","root","root","taiwan")
#連接
cursor = db.cursor()

In [ ]:
import csv
for count,i in enumerate(number[1:2]):
    #創建表格
    sql_createTb = """CREATE TABLE %s_TW (
                 Date  char(30) NOT NULL ,
                 High  char(30),
                 Low  char(30),
                 Open  char(30),
                 Close  char(30),
                 Volume  char(30),
                 Adj_Close  char(30),
                 PRIMARY KEY(Date))
                 """ % i
    cursor.execute(sql_createTb)
    db.commit()

    data = web.DataReader('%s.TW' % i, 'yahoo', start, end)
    data.to_csv('股票\%s.TW.csv' % i,index=True,sep=',')
    
    data = pd.read_csv('股票\%s.TW.csv' % i)  
    for j in range(data.shape[0]):
        sql_insert_temp = ""
        sql_insert = ""
        for z in range(data.shape[1]):
            sql_insert_temp +=  "'%s'" % data.iloc[j,z]
            if z < (data.shape[1] - 1):
                sql_insert_temp += ","
            #插入一条数据到moneytb 里面。
        sql_insert = "insert into taiwan.%s_TW values(%s)" % (i,sql_insert_temp) 
        #執行SQL
        cursor.execute(sql_insert)
        db.commit()
    print(i,name[count] , "finish")

